In [1]:
push!(LOAD_PATH, pwd())
@everywhere using SIS,IM,PayloadGraph,PyPlot, Epidemics
import LightGraphs

In [2]:
function plot_schematic(n_n,c_r,N,k=N-1,plot_k=false)
    beta = 4.0/(c_r*n_n)
    alpha = (N*beta)/n_n
    println(N,alpha,beta)

    y_n, y_minus,y_plus,y_p,critical_determinant = get_parameters(N,alpha,beta)
    f(y) = alpha.*y.^2
    s(y) = f(y)./y - beta
    get_y_eff(y,k) = y.*(1 + (1-y)./(y.*k))
    get_s_eff(y,alpha,beta,k) = alpha*get_y_eff(y,k) - beta


    y_range = collect(0:y_p/1000:1.9*y_p)
    figure(1)
    plot(y_range,1.0./abs(N*s(y_range)),"-r",label=L"$\frac{1}{N|s(y)|}$")
    if plot_k
        plot(y_range,1.0./abs(N*get_s_eff(y_range,alpha,beta,k)),"-b",label=L"$\frac{1}{N|s(y_{eff})|}$")
    end
    plot(y_range,y_range,"-k",label=L"$y$")
    axvline(y_n,linestyle="--",label=L"$y_n$")
    axvline(y_p,linestyle="-",label=L"$y_p$")
    if y_minus > 0
        axvline(y_minus,linestyle="-.",label=L"$y_1$")
        axvline(y_plus,linestyle="-.",label=L"$y_2$")
    end
    ylim([0,1.9*y_p])
    legend(prop=Dict{Any,Any}("size"=>15),loc="upper right")
    xlabel(L"$y$",size=20)
    if plot_k
        #figure(1)
        #plot(y_range,get_s_eff(y_range,alpha,beta,k),"-b",label=L"$s_{eff}(y)$")
        #plot(y_range,s(y_range),"-r",label=L"$s(y)$")
        legend(prop=Dict{Any,Any}("size"=>20),loc="upper right")
        xlabel(L"$y$",size=20)
    end
end

function plot_schematics(N,n_n,alpha,beta)
    plot_reach = true
    pygui(true)
    close("all")
    dx = 2*n_n/N/200
    x = collect(1/N:dx:2*n_n/N)
    if plot_reach
        y = IM.P_reach(im,N,1.0/N,x)
        yk = IM.P_reach(imk,N,1.0/N,x)
        plotfn = loglog
        plotstr = "reach"
    else
        y = IM.P_fix(im,N,x)
        yk = IM.P_fix(imk,N,x)
        plotfn = plot
        plotstr = "fix"
    end
    figure(2,figsize=(8,5))
    plotfn(x,y,"-r",label=latexstring("P_{$(plotstr)}(y)"))
    plotfn(x,yk,"-b",label=latexstring("P_{$(plotstr)}(y_{eff})"))
    plotfn(x,1/N./x,"--k",label=latexstring("P_{$(plotstr),neutral}"))
    xlim([0,2*n_n/N])
    y_n, y_minus, y_plus, y_p,critical_determinant = get_parameters(N,alpha,beta)
    axvline(y_n,linestyle="--",color="b",label=L"y_n")
    axvline(y_minus,linestyle="-.",color="r",label=L"y_1")
    axvline(y_plus,linestyle="-.",color="r",label=L"y_2")
    axvline(y_p,linestyle="-",color="b",label=L"y_p")
    xlabel(L"y")
    ylabel(latexstring("P_{$(plotstr)}(y)"))
    legend(loc="best")
    title(latexstring("\$y_n = $(n_n/N), c_r = $c_r, N = $N\$"))
    #savefig("p_fix_y_n = $(n_n/N), c_r = $c_r, N = $N.png")

    figure(1,figsize=(8,5))
    plot_schematic(n_n,c_r,N,k,true)
    title(latexstring("\$y_n = $(n_n/N), c_r = $c_r, N = $N\$"))

#     figure(3)
#     plot(x,get_s_eff(x,alpha,beta,k),"-b",label=L"$s_{eff}(y)$")
#     plot(x,get_s(x,alpha,beta),"-r",label=L"$s(y)$")
#     grid(1)

end

plot_schematics (generic function with 1 method)

Notes on P_fix:

- P_fix follows a logistical shape with the point of maximum slope = y_n.
- given $n_n$, it is independent of $N$. The larger $n_n$, the sharper the transition.
- the smaller $c_r < 1$, the sharper the transition at $y_n$. The larger $c_r < 1$, the shallower the transition
- For $c_r > 1$, the growth is linear and reaches $1$ at about $y = y_p$

In [24]:

@everywhere begin

k = 4
#y_n = 0.1
c_r = 0.2
N = 10000
n_n = 4000#y_n*N
beta = 4.0/(c_r*n_n)
alpha = (N*beta)/n_n
println(N, ' ' ,alpha, ' ',beta)

im = InfectionModel(x -> 1 + alpha*x , x -> 1 + beta);
imk = InfectionModel(x -> 1 + beta + get_s_eff(x,alpha,beta,k) , x -> 1 + beta);

y_n, y_minus, y_plus, y_p,critical_determinant = get_parameters(N,alpha,beta)
end

10000 0.0125 0.005


In [40]:
plot_schematics(N,n_n,alpha,beta)

100000.01250.005


2016-09-14 23:36:57.419 julia[3671:181219] setCanCycle: is deprecated.  Please use setCollectionBehavior instead
2016-09-14 23:36:57.438 julia[3671:181219] setCanCycle: is deprecated.  Please use setCollectionBehavior instead
2016-09-14 23:36:58.927 julia[3671:181219] setCanCycle: is deprecated.  Please use setCollectionBehavior instead
2016-09-14 23:36:58.935 julia[3671:181219] setCanCycle: is deprecated.  Please use setCollectionBehavior instead


PyObject <matplotlib.text.Text object at 0x31f58d650>

In [38]:
runs_well_mixed = run_epidemics(100000, () -> run_epidemic_well_mixed(N,im,1.0));
runs_well_mixedk = run_epidemics(100000, () -> run_epidemic_well_mixed(N,imk,1.0));

In [41]:
xvals,pvals = get_p_reach(runs_well_mixed)
xvals /= N
figure(2)
loglog(xvals,pvals,"-r")
xvals,pvals = get_p_reach(runs_well_mixedk)
xvals /= N
figure(2)
loglog(xvals,pvals,"-b")

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x31ef3b950>

In [37]:
close("all")